In [1]:
import pandas as pd
from __future__ import annotations
from pathlib import Path
from itertools import chain

import Bio
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.AlignInfo import SummaryInfo
from Bio.Align import AlignInfo

import warnings
warnings.filterwarnings("ignore")
import numpy as np
from Bio.motifs import Motif
import glob 
import itertools
import matplotlib.pyplot as plt

from scipy import stats


# Plan:
1. Calculate percent identity of each position in all AD alignments
2. Calculate percent identity of each position in full TF alignments
3. Compare distributions

# 1. Calculate percent identity of each position in all AD alignments


In [51]:
counts_dfs = []

print(str(len(glob.glob("../data/zoonomia_toga_mca/all_AD_alignments/*"))) + " alignments")
i = 0

# For each AD alignment
for AD_alignment in glob.glob("../data/zoonomia_toga_mca/all_AD_alignments/*"):
    if i%25 == 0:
        print(i)
    i += 1
    
    # Get msa
    msa = AlignIO.read(AD_alignment, "fasta")
    
    # Get consensus sequence (>50% of positions have that AA)
    # summary_align = AlignInfo.SummaryInfo(msa)
    # consensus = summary_align.dumb_consensus(0.5)
    
    #Counting each AA
    motif = Motif("ACDEFGHIKLMNPQRSTVWYX-", msa.alignment)
    counts = motif.counts
    
    # Saving as dataframe in desired format
    counts_df = pd.DataFrame(counts).T
    #counts_df.columns = list(str(consensus))
    counts_dfs.append(counts_df)

656 alignments
0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650


In [53]:
# Getting mean percent identity per AA per AD
per_AD_mean_perc_ident = []

for count_df in counts_dfs:
    df = count_df.copy(deep = True)

    # Getting proportions
    df = df / df.sum()   

    # Keeping index AA and value of greatest percent identity per position
    df_percent_identities = pd.DataFrame({"AA" : df.idxmax(axis=0), "percent identity" : df.max(axis=0)})
    #display(df_percent_identities)
    
    # Calculating mean percent identity per AA
    mean_perc_idents = df_percent_identities.groupby("AA").mean()
    
    per_AD_mean_perc_ident.append(mean_perc_idents)

per_AD_mean_perc_ident = pd.concat(per_AD_mean_perc_ident)
per_AD_mean_perc_ident.groupby("AA").median().sort_values(by = "percent identity", ascending = False)

,percent identity
AA,
W,0.974576
Y,0.965106
F,0.964888
K,0.963494
C,0.962963
L,0.959507
H,0.956645
Q,0.955986
R,0.955852


In [54]:
# Wide dataframe where columns are positions in the AD consensus sequences, rows are the 20 AAs
combined_counts_df = pd.concat(counts_dfs, axis = 1)
# Converting to proportions
combined_percents_df = combined_counts_df / combined_counts_df.sum()
combined_percents_df

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,162
A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.991579,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.971602,0.000000
E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.971602,0.000000,0.000000,0.000000,0.000000,0.000000
F,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
H,0.993684,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.995789,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [55]:
percent_identities = pd.DataFrame({"AA" : combined_percents_df.idxmax(axis=0), "percent identity" : combined_percents_df.max(axis=0)})
percent_identities

,AA,percent identity
0,H,0.993684
1,M,0.995789
2,M,0.995789
3,Y,0.995789
4,P,0.989474
...,...,...
158,P,0.971602
159,S,0.971602
160,S,0.971602
161,D,0.971602


# 2. Calculate percent identity of each position in full TF alignments

In [63]:
full_TF_counts_dfs = []

all_prot_paths = set(glob.glob("../data/zoonomia_toga_mca/prot_alignments/*")) | set(glob.glob("../data/zoonomia_toga_mca/prot_alignments_wrong_isoforms/*"))

print(str(len(all_prot_paths)) + " alignments")
i = 0

# For each AD alignment
for TF_alignment in all_prot_paths:
    if i%25 == 0:
        print(i)
    i += 1
    
    # Get msa
    msa = AlignIO.read(TF_alignment, "fasta")
    
    # Get consensus sequence (>50% of positions have that AA)
    # summary_align = AlignInfo.SummaryInfo(msa)
    # consensus = summary_align.dumb_consensus(0.5)
    
    #Counting each AA
    motif = Motif("ACDEFGHIKLMNPQRSTVWYX-", msa.alignment)
    TF_counts = motif.counts
    
    # Saving as dataframe in desired format
    TF_counts_df = pd.DataFrame(TF_counts).T
    #counts_df.columns = list(str(consensus))
    full_TF_counts_dfs.append(TF_counts_df)

541 alignments
0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525


In [71]:
combined_percents_df.iloc[:, 0].isin(TF_combined_percents_df.columns)

A     True
C     True
D     True
E     True
F     True
G     True
H    False
I     True
K     True
L     True
M     True
N     True
P     True
Q     True
R     True
S     True
T     True
V     True
W     True
Y     True
X    False
-    False
Name: 0, dtype: bool

In [64]:
# Wide dataframe where columns are positions in the AD consensus sequences, rows are the 20 AAs
TF_combined_counts_df = pd.concat(full_TF_counts_dfs, axis = 1)
# Converting to proportions
TF_combined_percents_df = TF_combined_counts_df / TF_combined_counts_df.sum()
TF_combined_percents_df

,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
A,0.000000,0.000000,0.000000,0.000000,0.000000,0.778017,0.221983,0.480603,0.183190,0.002155,...,0.002096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997904,0.000000
C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.209052,0.010776,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
E,0.000000,0.000000,0.000000,0.000000,0.000000,0.006466,0.411638,0.181034,0.038793,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
F,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
G,0.000000,0.000000,0.000000,0.000000,0.000000,0.010776,0.038793,0.012931,0.420259,0.000000,...,0.000000,0.000000,0.995807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006466,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002155,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002155,0.002155,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
L,0.000000,0.000000,0.000000,0.008621,0.153017,0.000000,0.000000,0.000000,0.015086,0.000000,...,0.000000,0.989518,0.000000,0.000000,0.000000,0.000000,0.995807,0.997904,0.000000,0.997904


In [65]:
# Remove positions which are in AD table

In [95]:
TF_percent_identities = pd.DataFrame({"AA" : TF_combined_percents_df.idxmax(axis=0), "percent identity" : TF_combined_percents_df.max(axis=0)})
TF_percent_identities = TF_percent_identities.reset_index(drop = True)
TF_percent_identities

,AA,percent identity
0,-,0.995690
1,-,0.997845
2,-,0.993534
3,M,0.909483
4,P,0.715517
...,...,...
439479,T,0.989518
439480,L,0.995807
439481,L,0.997904
439482,A,0.997904


In [97]:
percent_identities = percent_identities.reset_index(drop = True)
percent_identities

,AA,percent identity
0,H,0.993684
1,M,0.995789
2,M,0.995789
3,Y,0.995789
4,P,0.989474
...,...,...
79051,P,0.971602
79052,S,0.971602
79053,S,0.971602
79054,D,0.971602


In [101]:
idx_to_drop = []


for i in percent_identities.index:
    if i%100 == 0:
        print(i)
        
    # Get all matches in TF percent identities using AA and value
    merged_df = pd.merge(percent_identities.iloc[[i]], TF_percent_identities)
    
    if len(merged_df) != 0:
        # Only save index of first row to drop
        idx_to_drop.append(merged_df.index[0])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
len(idx_to_drop) # 

In [73]:
percent_identities

,AA,percent identity
0,H,0.993684
1,M,0.995789
2,M,0.995789
3,Y,0.995789
4,P,0.989474
...,...,...
158,P,0.971602
159,S,0.971602
160,S,0.971602
161,D,0.971602


In [76]:
pd.merge(TF_percent_identities, percent_identities, on = ["AA", "percent identity"], how = "inner")

,AA,percent identity
0,A,0.480603
1,Q,0.812500
2,A,0.696121
3,M,0.894397
4,W,0.924569
...,...,...
2994616,L,0.997904
2994617,L,0.997904
2994618,L,0.997904
2994619,L,0.997904
